In [2]:
%pip install -q duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import duckdb

# 다운로드한 Parquet 파일 경로
file_path = './beauty.parquet'

# DuckDB에 연결
con = duckdb.connect()

# Parquet 파일의 스키마(컬럼 정보)를 조회하여 데이터프레임으로 변환
try:
    columns_df = con.execute(f"DESCRIBE SELECT * FROM '{file_path}'").df()
    
    # 컬럼 이름만 리스트로 추출
    column_names = columns_df['column_name'].tolist()
    
    print("--- 파일 컬럼 리스트 ---")
    print(column_names)

except duckdb.Error as e:
    print(f"Error: {e}")
    print("파일 경로를 다시 확인해주세요.")

# 연결 종료
con.close()

--- 파일 컬럼 리스트 ---
['additives_n', 'additives_tags', 'allergens_tags', 'brands_tags', 'brands', 'categories', 'categories_tags', 'checkers_tags', 'cities_tags', 'code', 'complete', 'completeness', 'correctors_tags', 'countries_tags', 'created_t', 'creator', 'data_quality_errors_tags', 'data_quality_info_tags', 'data_quality_warnings_tags', 'data_sources_tags', 'editors', 'emb_codes_tags', 'emb_codes', 'entry_dates_tags', 'generic_name', 'images', 'informers_tags', 'ingredients_analysis_tags', 'ingredients_from_palm_oil_n', 'ingredients_n', 'ingredients_original_tags', 'ingredients_percent_analysis', 'ingredients_tags', 'ingredients_text', 'ingredients_with_specified_percent_n', 'ingredients_with_unspecified_percent_n', 'ingredients', 'known_ingredients_n', 'labels_tags', 'labels', 'lang', 'languages_tags', 'last_edit_dates_tags', 'last_editor', 'last_image_t', 'last_modified_by', 'last_modified_t', 'last_updated_t', 'link', 'main_countries_tags', 'manufacturing_places_tags', 'manufactur

In [16]:
import duckdb
import pandas as pd

# 다운로드한 Parquet 파일 경로
file_path = './beauty.parquet'

# DuckDB에 연결
con = duckdb.connect()

# 스킨케어 관련 키워드 목록 (언어와 무관)
skincare_keywords = [
    'skincare', 'face-care', 'face', 'serum', 'moisturizer', 'cream', 'creme', 
    'lotion', 'toner', 'tonic', 'gel', 'cleanser', 'wash', 
    'mask', 'scrub', 'exfoliator', 'peeling', 'sunscreen'
]

try:
    # SQL WHERE 절 동적 생성
    # array_to_string으로 배열을 문자열로 변환하고 LIKE를 사용
    where_conditions = " OR ".join([f"lower(array_to_string(categories_tags, ' ')) LIKE '%{keyword}%'" for keyword in skincare_keywords])

    # SQL 쿼리를 사용하여 조건에 맞는 제품 목록을 가져오기
    # 모든 데이터를 가져오기 위해 LIMIT 절을 사용하지 않습니다.
    query = f"""
        SELECT 
            code,
            product_name,
            brands,
            categories_tags,
            ingredients_text
        FROM '{file_path}'
        WHERE
            {where_conditions}
    """
    
    # 쿼리 결과를 pandas DataFrame으로 변환
    skincare_products_df = con.execute(query).df()

    print(f"--- 필터링된 모든 스킨케어 제품 목록 (총 {len(skincare_products_df)}개) ---")
    print(skincare_products_df)

except duckdb.Error as e:
    print(f"Error: {e}")
    print("쿼리문이 잘못되었거나, 파일에 해당 컬럼이 없는지 확인해주세요.")
    
finally:
    # 연결 종료
    con.close()

--- 필터링된 모든 스킨케어 제품 목록 (총 4320개) ---
               code                                       product_name  \
0     0010181032356  [{'lang': 'main', 'text': 'Huile de coco - crè...   
1     0010181040313  [{'lang': 'main', 'text': 'Lotion de massage A...   
2     0018787830338  [{'lang': 'main', 'text': 'Dr. Bronner's Bio K...   
3     0020714434878  [{'lang': 'main', 'text': 'Redness solutions'}...   
4     0022796170156  [{'lang': 'main', 'text': 'Shea butter hair ma...   
...             ...                                                ...   
4315  0840749000578  [{'lang': 'main', 'text': 'Revitalizing Eye Tr...   
4316  0850020056392  [{'lang': 'main', 'text': 'Super Clear Blemish...   
4317  0554392215615  [{'lang': 'main', 'text': 'sublime hair sérum'...   
4318  4001638589727                                                 []   
4319  8901030535697  [{'lang': 'main', 'text': 'Vim Dishwash Bar'},...   

                  brands                                    categories_tag

In [12]:
import pandas as pd

# 다운로드한 CSV 파일 경로를 지정해 주세요
file_path = './skincare_products_clean.csv' 

try:
    # CSV 파일을 pandas DataFrame으로 읽어오기
    df = pd.read_csv(file_path)

    # 데이터프레임의 상위 5개 행과 컬럼 정보 출력
    print("--- 파일 불러오기 성공 ---")
    print(df.head())
    
    # 컬럼 목록만 출력
    print("\n--- 데이터프레임 컬럼 목록 ---")
    print(df.columns)
    
except FileNotFoundError:
    print(f"Error: {file_path} 경로에 파일이 없습니다. 파일을 올바른 위치에 저장했는지 확인해 주세요.")
except Exception as e:
    print(f"Error: 파일을 읽어오는 중 오류가 발생했습니다. {e}")

--- 파일 불러오기 성공 ---
                                        product_name  \
0  The Ordinary Natural Moisturising Factors + HA...   
1      CeraVe Facial Moisturising Lotion SPF 25 52ml   
2  The Ordinary Hyaluronic Acid 2% + B5 Hydration...   
3          AMELIORATE Transforming Body Lotion 200ml   
4                     CeraVe Moisturising Cream 454g   

                                         product_url product_type  \
0  https://www.lookfantastic.com/the-ordinary-nat...  Moisturiser   
1  https://www.lookfantastic.com/cerave-facial-mo...  Moisturiser   
2  https://www.lookfantastic.com/the-ordinary-hya...  Moisturiser   
3  https://www.lookfantastic.com/ameliorate-trans...  Moisturiser   
4  https://www.lookfantastic.com/cerave-moisturis...  Moisturiser   

                                       clean_ingreds   price  
0  ['capric triglyceride', 'cetyl alcohol', 'prop...   £5.20  
1  ['homosalate', 'glycerin', 'octocrylene', 'eth...  £13.00  
2  ['sodium hyaluronate', 'sodium hyalur